Import data from wiki

In [189]:
#!conda install -c conda-forge requests --yes
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [216]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

In [217]:
My_table = soup.find('table',{'class':'wikitable sortable'})
#My_table

Get dataframe with wiki data

In [218]:
import pandas as pd
my_list=[]
str_postcode=[]
for row in My_table.findAll("tr"):
    cells = row.findAll("td")
    #For each "tr", assign each "td" to a variable.
    if len(cells) == 3:
        postcode = cells[0].find(text=True)
        borough = cells[1].find(text=True)
        neighbourhood = cells[2].find(text=True)
        my_list.append([postcode,borough,neighbourhood])
df=pd.DataFrame(my_list,columns=['PostCode','Borough','Neighbourhood'])
df.head()

,PostCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Clean data

In [219]:
#delete row where Borough is 'Not assigned'
df_cleaned=df[df['Borough']!='Not assigned']
#print (df_cleaned.head())
#print (df_cleaned.shape)
#df_cleaned

#replace 'Not assigned' of neighbourhood with borough value
#print (df_cleaned[df_cleaned['Neighbourhood']=='Not assigned'])
#print (df_cleaned[df_cleaned['PostCode']=='M7A'])
df_cleaned.at[8,'Neighbourhood'] = "Queen's Park"
#df_cleaned[df_cleaned['PostCode']=='M7A']

In [220]:
df_final=df_cleaned.groupby('PostCode')['Neighbourhood'].apply(', '.join).reset_index()
df_final = df_final.replace('\n','', regex=True)
df_final=df_final.merge(df_cleaned[['PostCode','Borough']],how='left',on='PostCode')
df_final = df_final.drop_duplicates(subset='PostCode').reset_index(drop=True)
#change sequence of columns
columnsTitles = ['PostCode', 'Borough', 'Neighbourhood']
df_final = df_final.reindex(columns=columnsTitles)
df_final.head()

,PostCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Get size of dataframe

In [221]:
df_final.shape

(103, 3)

Read geoinfo from csv file and combine it with the above dataframe

In [222]:
geo_info=pd.read_csv('Geospatial_Coordinates.csv')
geo_info.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Show the final dataframe

In [223]:
my_df = df_final.merge(geo_info,how='left',left_on='PostCode',right_on='Postal Code')
my_df=my_df.drop(columns='Postal Code')
#print (my_df[my_df['PostCode']=='M9V'])
my_df.head()

,PostCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
